#### start this notebook on 

- orion partition, 
- normal queue,
- 1 node, 
- 40 tasks, 
- --exclusive for additional slurm parameters
- and don't forget to type your conda env path in the additional kernel locations box

#### conda packages needed

dask, distributed, xarray, rioxarray, rasterio, numpy, pandas, xesmf, netcdf4, glob2, matplotlib, ipykernel


---
# Things from 2024-03-13 meeting
---

# Reproject bounding box

this section runs faster using dask single-machine scheduler (the default) instead of dask distributed (where we would set up a cluster/client)

In [ ]:
# packages
import xarray as xr
import rioxarray as rio

In [ ]:
# paths
basedir='/our/shared/datasets/dir/'
path_to_raw_data = 'raw_data/'
path_to_climatic = basedir + path_to_raw_data + 'ERA5/month/'

path_to_pre = path_to_climatic + 'era5_monthly_means_50_21_pre.nc'
path_to_atlas_ai=basedir + path_to_raw_data + 'ATLAS-AI project/ATLAS-AI/AWI/2022_AWI_43-SSA-contiguous_AWI_43-SSA_2003.tif'

# btw, I highly recommend replacing the space with an underscore in the dir "ATLAS-AS project"
# spaces in dir and file names can cause problems in a linux operating system

In [ ]:
# AWI
atlas_ai=xr.open_dataset(path_to_atlas_ai)
atlas_ai_unproj=atlas_ai.rio.reproject(4326) # reproject the whole thing
minx,miny,maxx,maxy = atlas_ai_unproj.rio.bounds() # now get the bounds
print(minx,miny,maxx,maxy)
atlas_ai_unproj

#### notice the longitude coordinates in atlas_ai_unproj are on the -180 to 180 scale not 0 to 360 scale

#### if we plot the original data atlas_ai and the reprojected data atlas_ai_unproject we can see the reprojection worked

In [ ]:
# plot
atlas_ai.band_data.plot(cmap='RdBu')

In [ ]:
# plot
atlas_ai_unproj.band_data.plot(cmap='RdBu')


In [ ]:
del atlas_ai, atlas_ai_unproj # free up RAM

#### it looks like part of the top of Africa is chopped off even in the original AWI data file, so the reprojection has the same extents

#### now look at precipitation

In [ ]:
# let it auto chunk
pre_world = xr.open_dataarray(path_to_pre, decode_coords = "all", chunks='auto')#{'time':-1,'latitude':-1,'longitude':60})
pre_world

#### pre_world longitudes are 0 to 360. this is what was causing the problem clipping part of Africa incorrectly

#### to fix the problem, just re-write the longitude coordinate to -180 to 180

#### this is something that should be done for every input dataset as part of the data cleaning process anyways

In [ ]:
lon_attrs=pre_world.longitude.attrs # copy metadata
pre_world.coords['longitude']=xr.where(pre_world.coords['longitude']>180,pre_world.coords['longitude']-360,pre_world.coords['longitude'])
pre_world=pre_world.roll(longitude=1799,roll_coords=True)
pre_world['longitude'].attrs=lon_attrs
pre_world.rio.write_crs(4326, inplace = True)
pre_world

In [ ]:
%%time
# this will crash your notebook if you aren't running on enough cores for this ~21GB data variable
# the fewer tasks you've launched the jupyter session with, the slower this will run
pre_world=pre_world.compute()

In [ ]:
pre_world.isel(time=0).plot()

In [ ]:
pre = pre_world.rio.clip_box(minx,miny,maxx,maxy)
pre.isel(time=0).plot()

#### since the atlas_ai variable doesn't include the northernmost part of Africa this clip looks like it worked correctly

---
### rewrite netcdf files to change dimension/coordinate names

In [ ]:
import xarray as xr

In [ ]:
# use dask distributed, starting a LocalCluster
# we are using dask.distributed LocalCluster because I couldn't get this to work with regular dask 
# (default is the single-machine scheduler where you don't have to start a LocalCluster)

# if you started the notebook on 1 node, 40 tasks, you will have 80 threads to work with
# the notebook itself is also running on 1 (or likely more than 1) of these threads
# usually when I start a local cluster I'll start it on a few less than 80 to try to avoid crashing the notebook
# Full disclosure, I'm not 100% sure this technique of using slightly less threads does what I think

from dask.distributed import Client,LocalCluster

nworkers=72 # I'm choosing a few less than 80  
cluster=LocalCluster(n_workers=nworkers,threads_per_worker=1) # a cluster where each thread is a separate process or "worker"
client=Client(cluster)  # connect to your compute cluster
client.wait_for_workers(n_workers=nworkers,timeout=10) # wait up to 10s for the cluster to be fully ready, error if not ready in 10s
client # print info

In [ ]:
# something weird is going on with the ERA5 file when time gets decoded
# so first, don't decode time
ds=xr.open_dataset(path_to_pre,chunks='auto',decode_times=False,lock=False).rename({'latitude':'lat','longitude':'lon'})

# then, copy the attributes from the un-decoded time variable
# we'll use these later when we re-write the netcdf
calendar=ds.time.attrs['calendar']
units=ds.time.attrs['units']

# then, decode time with the cftime library
ds=xr.decode_cf(ds,use_cftime=True)
ds=ds.transpose('lat','lon','time') # reorder dimensions

# change precip units
ds['tp']=ds.tp*1000. # m-->mm
ds.tp.attrs['units']='mm'
ds

In [ ]:
# new file path and name
outdir=basedir+'temporary/'
prfile=outdir+'era5_monthly_means_50_21_pre.nc'

In [ ]:
%%time

# the encodings below allow you to keep the same dimension/coordinate attributes from the original file
# we are essentially overriding some xarray default settings here
time_encoding={'_FillValue':None,'calendar':calendar,'units':units} 
lat_encoding={'_FillValue':None}
lon_encoding={'_FillValue':None}

# compute and write new file in parallel
output=ds.to_netcdf(prfile,
            encoding={'lat':lat_encoding,
                      'lon':lon_encoding,
                      'time':time_encoding},
            compute=False)
output.compute()

In [ ]:
client.shutdown()

---
### run climate-indices command line tool

This section probably doesn't run, but I'm leaving it in here as example code. 

It would only work if you've installed the climate-indices package with pip (last step after building your conda env) and you're working with a self installed version of miniconda instead of the system module miniconda.

In [ ]:
import subprocess
import matplotlib.pyplot as plt

In [ ]:
# below works for me but probably won't for others
# this is because I'm using my own installation of miniconda instead of using the Orion system module conda
# we'll probably have to go through it together and modify a few things to get it working 

# user conda information
user_home='$HOME/'
conda_init_file=user_home+'miniconda3/etc/profile.d/conda.sh'  # you need to find your conda.sh script
conda_env='/workspace/username/envs/dataprep' # path user-specific conda env

# create command enclosed in ""
# everything but the final command should end with ;
command_conda_init=". "+conda_init_file+";"               # initialize conda
command_env_activate="conda activate "+conda_env+";"      # activate conda environment 
command_echo_env="echo 'conda env =' $CONDA_DEFAULT_ENV;" # print active env name
command_spi="process_climate_indices "+\
                "--index spi "+\
                "--periodicity monthly "+\
                "--netcdf_precip "+prfile+" "\
                "--var_name_precip tp "+\
                "--output_file_base "+outdir+" "\
                "--scales 6 12 "+\
                "--calibration_start_year 1951 "+\
                "--calibration_end_year 2021 "+\
                "--multiprocessing all"

In [ ]:
# double check spaces are all good in command_spi
command_spi

In [ ]:
# string all the commands together
# climate-indices commands take a very long time, this took 30 minutes to run
# also you get a lot of divide by zero and nan runtime warning and unless 
# the climate-indices source is modified, I don't think there is any way to silence 
# these when calling from python like this

subprocess.run(command_conda_init+
               command_env_activate+
               command_echo_env+
               command_spi,
               shell=True)

In [ ]:
# take a look at the results
ds_g06=xr.open_dataset(basedir+'temporary/_spi_gamma_06.nc')
ds_g12=xr.open_dataset(basedir+'temporary/_spi_gamma_12.nc')
ds_p06=xr.open_dataset(basedir+'temporary/_spi_pearson_06.nc')
ds_p12=xr.open_dataset(basedir+'temporary/_spi_pearson_12.nc')
ds_g06

In [ ]:
# 6-month spi so first 5 time indexes will be nan
# import cartopy
fig=plt.figure(figsize=(15,10))

ax=fig.add_subplot(221)
ds_g06.spi_gamma_06.isel(time=11).plot(ax=ax)

ax=fig.add_subplot(222)
ds_g12.spi_gamma_12.isel(time=11).plot(ax=ax)


ax=fig.add_subplot(223)
ds_p06.spi_pearson_06.isel(time=11).plot(ax=ax)

ax=fig.add_subplot(224)
ds_p12.spi_pearson_12.isel(time=11).plot(ax=ax)

In [ ]:
fig=plt.figure(figsize=(15,2))
y1=ds_g06.spi_gamma_06.sel(lat=0,lon=25,method='nearest')  # gamma in blue
y2=ds_p06.spi_pearson_06.sel(lat=0,lon=25,method='nearest') # pearson in orange
y1.plot()
y2.plot()
plt.axhline(y=0,color='grey',linestyle='--')
plt.show()


fig=plt.figure(figsize=(15,2))
y1=ds_g12.spi_gamma_12.sel(lat=0,lon=25,method='nearest')  # gamma in blue
y2=ds_p12.spi_pearson_12.sel(lat=0,lon=25,method='nearest') # pearson in orange
y1.plot()
y2.plot()
plt.axhline(y=0,color='grey',linestyle='--')
plt.show()

---
# Things from 2024-03-20 meeting
---
- aggregate tif data to netcdf with time dim

In [ ]:
# general workflow

# 1) fix up metadata before writing to netcdf
# atlas_ai=atlas_ai.squeeze()
# del atlas_ai.coords['band']
# atlas_ai=atlas_ai.rename({'band_data':'AWI'})

# 2) create time dim

# 3) create netcdf encodings

# 4) write netcdf

In [ ]:
# how to open multiple files at once
# xr.open_mfdataset()

---
# Things from 2024-03-27 meeting
---

### xesmf area conservative regridding and comparing to rasterio sum, bilinear

Here I use some different methods than what is in prepare_data.ipynb to get filenames, years, and I/O of data. It may not be any better or faster than your prepare_data.ipynb code, I just want to show you some other options.

I'm regridding from the pop grid to the ERA5 grid as you were doing in prepare_data.ipynb. But as we talked about, I think you should probably make the population grid your common grid and regrid all your other data to it. This way you don't have to worry about how regridding affects your population variable and any other variables that are computed with it. Bilinear "upsampling" of climate data (going from coarser to finer resolution) is standard practice and shouldn't cause problems in a peer-review.

In [ ]:
import glob
import numpy as np
import xarray as xr
import rioxarray as rio
import pandas as pd
import xesmf as xe
import matplotlib.pyplot as plt
from rasterio.enums import Resampling
import matplotlib

In [ ]:
# I'm going chunk the datasets and use dask to speed up the computing
# so start a local cluster

from dask.distributed import Client,LocalCluster

nworkers=40   
cluster=LocalCluster(n_workers=nworkers,threads_per_worker=1) # a cluster where each thread is a separate process or "worker"
client=Client(cluster)  # connect to your compute cluster
client.wait_for_workers(n_workers=nworkers,timeout=10) # wait up to 10s for the cluster to be fully ready, error if not ready in 10s
client # print info

In [ ]:
basedir='/path/to/our/shared/datasets/dir/'
path_to_raw_data = basedir+'raw_data/'
path_to_processed_data = basedir+'processed_data/'

path_to_wealth = path_to_raw_data + 'ATLAS/wealth/'
path_to_pop = path_to_raw_data + 'ATLAS/population/'

# 'auto' for chunksize produces 4 chunks per pop/wealth data file
# {'x':500,'y':-1} for chunksize produces 20 chunks per pop/wealth data file
# {'x':500,'y':-1} turns out to go a few seconds faster than 'auto' for each .compute()
chunksize={'x':500,'y':-1} # 'auto'

In [ ]:
# a list of all wealth and all pop files, sorted by year ascending
wealth_files=sorted(glob.glob(path_to_wealth + '*.tif'))
pop_files = sorted(glob.glob(path_to_pop + '*POP_43-SSA_*.tif'))

# create list of years from file names
years_wealth=[f[-8:-4] for f in wealth_files] # this is a python list comprehension
years_pop=[f[-8:-4] for f in pop_files]

# check there are the same number of files and same years in the wealth and pop file names
assert len(wealth_files)==len(pop_files), f'found {len(wealth_files)} wealth files, {len(pop_files)} pop files'
assert years_wealth==years_pop, f'found different years of wealth and pop files'

In [ ]:
# create datetimes
dates=pd.date_range(years_wealth[0],years_wealth[-1],freq='YS')
dates

In [ ]:
# do the minimum amount of processing to get all data years into an xr.dataArray
wealth_list=[]
for iy,f in enumerate(wealth_files):
    # xarray instead of rio will do the nan replacement for you
    # squeeze out the singleton dim 'band' when reading
    # the data variable will be automatically named 'band_data' in the dataset
    # here I'm putting each file into a dask array with 20 chunks
    # chunking signals to hold nothing in memory and to make computations lazy until we call compute
    
    data=xr.open_dataset(f).squeeze().chunk(chunksize)
    wealth_list.append(data)
    
# now we have a list of datasets in the correct time order, nothing held in memory
# look at one item in the list
wealth_list[0]

In [ ]:
%%time
# concatenate list of arrays on a new time dimension
wealth_ds=xr.concat(wealth_list,dim='time')
wealth_ds=wealth_ds.compute()
wealth_ds

In [ ]:
# add time coordinate info and clean up other metadata
wealth_ds=wealth_ds.assign_coords({'time':dates})
del wealth_ds.coords['band']
wealth_ds=wealth_ds.rename({'band_data':'wealth_pc'})
wealth_ds

In [ ]:
%%time
# do the same for population
pop_list=[]
for iy,f in enumerate(pop_files):
    pop_list.append(xr.open_dataset(f).squeeze().chunk(chunksize))    

pop_ds=xr.concat(pop_list,dim='time')
pop_ds=pop_ds.compute()

pop_ds=pop_ds.assign_coords({'time':dates})
del pop_ds.coords['band']
pop_ds=pop_ds.rename({'band_data':'pop'})
pop_ds  

In [ ]:
# check metadata is the same in both datasets
assert all(wealth_ds.x==pop_ds.x), f'pop and wealth have different longitudes'
assert all(wealth_ds.y==pop_ds.y), f'pop and wealth have different latitudes'
assert wealth_ds.spatial_ref.attrs==pop_ds.spatial_ref.attrs, f'pop and wealth have different spatial references'
assert wealth_ds.wealth_pc.attrs==pop_ds.pop.attrs, f'pop and wealth have different attributes'
assert wealth_ds.wealth_pc.dtype==pop_ds.pop.dtype, f'pop has dtype {pop_ds.pop.dtype}, wealth has dtype {wealth_ds.wealth_pc.dtype}'

In [ ]:
# no errors, we're safe to combine variables into one dataset
ds=wealth_ds.copy()
ds['pop']=pop_ds.pop
ds['wealth']=ds.pop*ds.wealth_pc

print(ds.rio.crs)
ds

In [ ]:
%%time

# reproject the whole thing
pop_3857=ds.pop
pop_4326=pop_3857.rio.reproject('EPSG:4326')

# now subset to a test area
# define tiny area in North Africa
# the lat/lons of the area we want after regridding
latN_NA,latS_NA=19.75,19.5
lonW_NA,lonE_NA=29.0,29.25
# the lat/lons of the area to use for regridding 
# (which includes a buffer based on the target grid resolution)
buff=0.1
latN_NA_clip,latS_NA_clip=latN_NA+buff,latS_NA-buff
lonW_NA_clip,lonE_NA_clip=lonW_NA-buff,lonE_NA+buff

# subset
# contains the area we want to regrid plus a buffer so we don't get nan on regridded edge pixels
pop_4326_NA=pop_4326.sel(y=slice(latN_NA_clip,latS_NA_clip),x=slice(lonW_NA_clip,lonE_NA_clip))
pop_4326_NA.rio.write_crs(4326, inplace = True)
print(pop_4326_NA.shape)

# 4326 bounds on the pop grid (includes buffer)
minx,miny,maxx,maxy = pop_4326_NA.rio.bounds()
print(minx,miny,maxx,maxy)

In [ ]:
# plot all of Africa projection 3857 vs 4326
levels=[2,5,10,20,30,40,50,60,70,80,90,100,250,500,750,1000,5000,10000,50000]
cmap='tab20'
cbar_kwargs={'ticks':levels}

fig = plt.figure(figsize = (20, 10))
ax = fig.add_subplot(121) 
pop_3857.isel(time=0).plot(ax = ax, levels=levels,cmap=cmap,cbar_kwargs=cbar_kwargs)
plt.title('epsg3857, original pop grid')

ax = fig.add_subplot(122) 
pop_4326.isel(time=0).plot(ax = ax, levels=levels,cmap=cmap,cbar_kwargs=cbar_kwargs)
plt.title('epsg4326, original pop grid')

plt.tight_layout()
plt.show()

### look at regridding methods over a tiny area in North Africa

In [ ]:
# show the tiny area on a larger map (it's at the top on the blue square of data)

fig = plt.figure(figsize = (10, 4))
ax = fig.add_subplot(111)

# plot same 4326 pop data as above, subset in latitude
pop_4326.sel(time='2003',y=slice(20,4)).plot(ax = ax, levels=levels,cmap=cmap,cbar_kwargs=cbar_kwargs)
plt.title('epsg4326, original pop grid')

# add a black rectangle based on the clip coordinates
rect = plt.Rectangle(xy=(lonW_NA_clip,latS_NA_clip), 
                     width=(lonE_NA_clip-lonW_NA_clip), 
                     height=(latN_NA_clip-latS_NA_clip),
                     facecolor=None,
                     edgecolor="black")
ax.add_patch(rect)

plt.show()

In [ ]:
# target grid for tiny North Africa
path_to_pre = path_to_raw_data + 'ERA5/month/era5_monthly_means_50_21_pre.nc'
pre = rio.open_rasterio(path_to_pre).sel(time='2003-01-01').squeeze() # getting a single time
del pre.coords['time']
pre = pre.assign_coords(x = (((pre.x + 180) % 360) - 180)).sortby('x') # fix longitudes
pre.rio.write_crs(4326, inplace = True) # add crs info
target_grid = pre.rio.clip_box(lonW_NA,latS_NA,lonE_NA,latN_NA) # target grid boundaries fully inside of clipped area

# 3857 bounds of target grid (fully inside of clipped area)
minx_3857,miny_3857,maxx_3857,maxy_3857 = target_grid.rio.reproject('EPSG:3857').rio.bounds() 
print(minx_3857,miny_3857,maxx_3857,maxy_3857)

# the data in target_grid doesn't matter
# xesmf will only use its coordinate and crs info for regridding population
target_grid

In [ ]:
# data to regrid
data_in=pop_4326_NA.sel(time='2003') # apparently it's a known issue that rioxarray drops the crs often
data_in.rio.write_crs("epsg:4326", inplace = True) # so re-write it
data_in.y.attrs={'standard_name':'latitude'} # dim names x,y with no associated label 'latitude','longitude' can cause problems
data_in.x.attrs={'standard_name':'longitude'} # so add this metadata
print(data_in.shape, target_grid.shape)
print(data_in.rio.crs)
data_in

In [ ]:
# rioxarray regrid
# rio bilinear
pop_4326_rio_bilinear = data_in.rio.reproject_match(target_grid, resampling = Resampling.bilinear)
# rio sum
pop_4326_rio_sum = data_in.rio.reproject_match(target_grid, resampling = Resampling.sum)

In [ ]:
# xesmf regrid
# set up the regridder functions
regridder_bilinear=xe.Regridder(data_in,target_grid,'bilinear',unmapped_to_nan=True) # calc weights
regridder_conservative=xe.Regridder(data_in,target_grid,'conservative',unmapped_to_nan=True) # calc weights
print(regridder_bilinear)
print('----------------------------------------')
print(regridder_conservative)

# execute the regridding
print('executing bilinear regrid...')
result_bilinear=regridder_bilinear(data_in,skipna=True,na_thres=1.) # apply weights
print('executing conservative regrid...')
result_conservative=regridder_conservative(data_in,skipna=True,na_thres=1.) # apply weights

#### idk what that warning is about, it's a new one for me

In [ ]:
# put all these things in a single dataset (just for fun)
results_ds=result_bilinear.to_dataset(name='pop_4326_xe_bilinear')
results_ds['pop_4326_xe_conservative']=result_conservative
results_ds['pop_4326_rio_bilinear']=pop_4326_rio_bilinear
results_ds['pop_4326_rio_sum']=pop_4326_rio_sum
results_ds

In [ ]:
# print the data ranges for the original data and each regrid method

# remember, the original 3857 data is clipped to a slightly larger area than our regridded data
print('pop_3857',
      pop_3857.rio.clip_box(minx_3857,miny_3857,maxx_3857,maxy_3857).min().data,
      pop_3857.rio.clip_box(minx_3857,miny_3857,maxx_3857,maxy_3857).max().data)

# print min/max for each variable in the dataset
for var in results_ds.keys():
    print(var, results_ds[var].min().data,results_ds[var].max().data)

#### in the next plot we want to force all our colorbars to have the exact same discrete values and colors, so we get the color names from the colormap in hex format, this way we can specific which color should go with which data values

In [ ]:
cmap = matplotlib.colormaps['tab20']
hexcolors=[matplotlib.colors.rgb2hex(cmap(i)) for i in range(cmap.N)]
hexcolors

In [ ]:
# obviously you would want Resampling.sum if regridding population from 
# fine to coarse resolution (which you should probably avoid doing anyway)
# but here is a picture of what all the regrid methods are doing
# edgecolor is what turns on the pixel boundaries

# plot
print('plotting...')
levels=[1,1.17,1.18,1.19,1.20,1.21] # custom levels for colorbar
colors=hexcolors[:len(levels)+1] # custom colors for colorbar
print(len(levels),len(colors)) # should be 1 more color than level

# figure with 5 subplots
fig = plt.figure(figsize = (30, 10))
ax = fig.add_subplot(151) 
cbar_kwargs={'ticks':levels,'extend':'max'} # I'm setting the extend part based on the min/max of each variable
pop_4326_NA.sel(time='2003',x=slice(28.95,29.35),y=slice(19.85,19.45)).plot(ax=ax,levels=levels,colors=colors,cbar_kwargs=cbar_kwargs,edgecolor='black')
plt.title('epsg4326, no regridding',fontsize='x-large')

ax = fig.add_subplot(152)
cbar_kwargs={'ticks':levels,'extend':'neither'} # I'm setting the extend part based on the min/max of each variable
results_ds.pop_4326_rio_bilinear.plot(ax = ax, levels=levels,colors=colors,cbar_kwargs=cbar_kwargs,edgecolor='black')
plt.title('epsg4326, rio bilinear regrid',fontsize='x-large')

ax = fig.add_subplot(153)
results_ds.pop_4326_xe_bilinear.plot(ax = ax, levels=levels,colors=colors,cbar_kwargs=cbar_kwargs,edgecolor='black')
plt.title('epsg4326, xe bilinear regrid',fontsize='x-large')

ax = fig.add_subplot(154) 
results_ds.pop_4326_xe_conservative.plot(ax = ax, levels=levels,colors=colors,cbar_kwargs=cbar_kwargs,edgecolor='black')
plt.title('epsg4326, xe conservative regrid',fontsize='x-large')

ax = fig.add_subplot(155)
# different levels and colors for this one
levels=[95,105,115,125,135,145,155]
colors=hexcolors[:len(levels)+1]
cbar_kwargs={'ticks':levels,'extend':'both'} # I'm setting the extend part based on the min/max of each variable
results_ds.pop_4326_rio_sum.plot(ax = ax, levels=levels,colors=colors,cbar_kwargs=cbar_kwargs,edgecolor='black')
plt.title('epsg4326, rio sum regrid',fontsize='x-large')

plt.tight_layout()
plt.show()

In [ ]:
client.shutdown()

---

---
# Things from 2024-05-29 meeting
---

- parallel rio.reproject_match

We need to think about parallelization in a different way because rio.reproject_match doesn't work on dask arrays. We won't load all files simultaneously as a dask array with 1 file per chunk. Instead we'll write a function that loads 1 file at a time as an xarray object and then use dask delayed to execute the reproject on all data files simultaneously.

Description of what will happen:
- write function "reprojmatch" that loads 1 data file as xarray object, reprojects, and returns an xarray object
- use dask delayed to build a list of delayed computational tasks
    - each task will be to execute our reprojmatch function and return the result for 1 data file
    - we have 11 data files, so there should be 11 delayed tasks
- execute all the delayed tasks simultaneously
    - tasks will be executed by different cores on orion
    - we'll get an xarray object returned from each executed task, collected into a list (a list of xarray objects)
- if we want a single data array that includes all our times (all data files) we can concat the list of results

In [1]:
# packages
import xarray as xr
import rioxarray as rio
from rasterio.enums import Resampling
import glob
import dask

In [2]:
basedir='/path/to/our/shared/datasets/dir/'

popfile=basedir+'raw_data/ATLAS/population/2022_POP_43-SSA-contiguous_POP_43-SSA_2003.tif'
lc_files=glob.glob(basedir+'raw_data/ERA5/dataset-satellite-land-cover/*.nc')

In [15]:
# notice these files aren't ordered in time
lc_files

In [4]:
# the target grid
pop=xr.open_dataarray(popfile).squeeze()
pop


<xarray.DataArray 'band_data' (y: 7673, x: 9902)>
[75978046 values with dtype=float32]
Coordinates:
    band         int64 1
  * x            (x) float64 -2.828e+06 -2.828e+06 ... 7.072e+06 7.072e+06
  * y            (y) float64 3.168e+06 3.167e+06 ... -4.503e+06 -4.504e+06
    spatial_ref  int64 ...
Attributes:
    AREA_OR_POINT:  Area

### first test reproject_match on 1 year

In [5]:
%%time

# open in memory, subset to Africa
lc = xr.open_dataset(lc_files[0]).sel(lat=slice(40,-40),lon=slice(-20,60))

# fix things for rio.reproject_match
lccs_class=lc.lccs_class
lccs_class=lccs_class.rio.write_crs(lc.crs) # crs is in the lc dataset (written as a variable instead of a spatial ref)
lccs_class=lccs_class.rename({'lat':'y','lon':'x'})

# reproject_match
test=lccs_class.rio.reproject_match(pop,resampling = Resampling.nearest)
test

CPU times: user 3.44 s, sys: 544 ms, total: 3.99 s
Wall time: 4.01 s


<xarray.DataArray 'lccs_class' (time: 1, y: 7673, x: 9902)>
array([[[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]]], dtype=uint8)
Coordinates:
  * x            (x) float64 -2.828e+06 -2.828e+06 ... 7.072e+06 7.072e+06
  * y            (y) float64 3.168e+06 3.167e+06 ... -4.503e+06 -4.504e+06
  * time         (time) datetime64[ns] 2016-01-01
    spatial_ref  int64 0
Attributes:
    standard_name:        land_cover_lccs
    flag_colors:          #ffff64 #ffff64 #ffff00 #aaf0f0 #dcf064 #c8c864 #00...
    long_name:            Land cover class defined in LCCS
    valid_min:            1
    valid_max:            220
    ancillary_variables:  processed_flag current_pixel_state observation_coun...
    flag_meanings:        no_data cropland_rainfed cropland_rainfed_herbaceou...
    flag_values:          [  0  10  11  12  20  30  40  50  60  61  62  70  7...
    _FillValue:           255

### how to get it working in parallel with dask delayed


In [6]:
# put everything in a function

def rio_reprojmatch_lc(f,target):
    # open in-memory and subset
    lc = xr.open_dataset(f).sel(lat=slice(40,-40),lon=slice(-20,60)) 
    
    # fix stuff
    lccs_class=lc.lccs_class
    lccs_class=lccs_class.rio.write_crs(lc.crs) 
    lccs_class=lccs_class.rename({'lat':'y','lon':'x'})    
    
    # reproject
    output=lccs_class.rio.reproject_match(target, resampling = Resampling.nearest)
    
    return output

In [7]:
# biuld a list of computational tasks, not executed yet
# we're creating 1 delayed compute task for each data file

task_list=[dask.delayed(rio_reprojmatch_lc)(f,pop) for f in lc_files]
task_list

[Delayed('rio_reprojmatch_lc-958673f8-ba10-466b-9630-c5bbd224f3be'),
 Delayed('rio_reprojmatch_lc-6cc6a9bc-f160-4d97-823a-e71e2e47131a'),
 Delayed('rio_reprojmatch_lc-a85b6837-541d-49d6-80af-2b4d1b2a52a9'),
 Delayed('rio_reprojmatch_lc-90391f3c-503c-4292-a743-9df8aca8543f'),
 Delayed('rio_reprojmatch_lc-1368b4fa-3710-4a17-be78-29f10da7fe7d'),
 Delayed('rio_reprojmatch_lc-16930b4c-6fa0-446f-afc4-b63d629d2572'),
 Delayed('rio_reprojmatch_lc-74cf3259-5405-4af0-906d-cbf0021d7fa6'),
 Delayed('rio_reprojmatch_lc-204da0c7-86e1-4163-adc9-af03d33aca23'),
 Delayed('rio_reprojmatch_lc-fd7e83e2-67b6-4f01-8320-83bb41fc7126'),
 Delayed('rio_reprojmatch_lc-322bfa91-c495-4d08-bdd2-e7b5cebcc417'),
 Delayed('rio_reprojmatch_lc-2eb87ffd-f7ca-47c7-92cb-7c3196d46442')]

In [9]:
%%time
# now execute all the compute tasks at the same time
result=dask.compute(*task_list)
result[0]

CPU times: user 39 s, sys: 6.23 s, total: 45.2 s
Wall time: 28.6 s


<xarray.DataArray 'lccs_class' (time: 1, y: 7673, x: 9902)>
array([[[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]]], dtype=uint8)
Coordinates:
  * x            (x) float64 -2.828e+06 -2.828e+06 ... 7.072e+06 7.072e+06
  * y            (y) float64 3.168e+06 3.167e+06 ... -4.503e+06 -4.504e+06
  * time         (time) datetime64[ns] 2016-01-01
    spatial_ref  int64 0
Attributes:
    standard_name:        land_cover_lccs
    flag_colors:          #ffff64 #ffff64 #ffff00 #aaf0f0 #dcf064 #c8c864 #00...
    long_name:            Land cover class defined in LCCS
    valid_min:            1
    valid_max:            220
    ancillary_variables:  processed_flag current_pixel_state observation_coun...
    flag_meanings:        no_data cropland_rainfed cropland_rainfed_herbaceou...
    flag_values:          [  0  10  11  12  20  30  40  50  60  61  62  70  7...
    _FillValue:           255

#### what's happening

if 1 file took about 4 seconds to load and reproject we would expect calculating all years simulateously to take just a little longer than that

if we calculated all years serially, like in a loop, we would expect 4s x 11 data files = about 44 seconds

But the above calculation of all years with dask delayed took about 28s. Faster than serial calculation, but much slower than expected. **Why?**  I'm not sure but I suspect is has something to do with using dask single-machine scheduler in an hpc environment.


Let's try using dask distributed LocalCluster

In [10]:
from dask.distributed import Client,LocalCluster

nworkers=11
cluster=LocalCluster(n_workers=nworkers,threads_per_worker=1) # a cluster where each thread is a separate process or "worker"
client=Client(cluster)  # connect to your compute cluster
client.wait_for_workers(n_workers=nworkers,timeout=10) # wait up to 10s for the cluster to be fully ready, error if not ready in 10s
client # print info

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 11
Total threads: 11,Total memory: 183.59 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35355,Workers: 11
Dashboard: http://127.0.0.1:8787/status,Total threads: 11
Started: Just now,Total memory: 183.59 GiB
Comm: tcp://127.0.0.1:39300,Total threads: 1
Dashboard: http://127.0.0.1:41398/status,Memory: 16.69 GiB
Nanny: tcp://127.0.0.1:39331,


In [12]:
%%time
task_list=[dask.delayed(rio_reprojmatch_lc)(f,pop) for f in lc_files]
result=dask.compute(*task_list)
result[0]

CPU times: user 836 ms, sys: 1.01 s, total: 1.85 s
Wall time: 6.14 s


<xarray.DataArray 'lccs_class' (time: 1, y: 7673, x: 9902)>
array([[[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]]], dtype=uint8)
Coordinates:
  * x            (x) float64 -2.828e+06 -2.828e+06 ... 7.072e+06 7.072e+06
  * y            (y) float64 3.168e+06 3.167e+06 ... -4.503e+06 -4.504e+06
  * time         (time) datetime64[ns] 2016-01-01
    spatial_ref  int64 0
Attributes:
    standard_name:        land_cover_lccs
    flag_colors:          #ffff64 #ffff64 #ffff00 #aaf0f0 #dcf064 #c8c864 #00...
    long_name:            Land cover class defined in LCCS
    valid_min:            1
    valid_max:            220
    ancillary_variables:  processed_flag current_pixel_state observation_coun...
    flag_meanings:        no_data cropland_rainfed cropland_rainfed_herbaceou...
    flag_values:          [  0  10  11  12  20  30  40  50  60  61  62  70  7...
    _FillValue:           255

**6 seconds! That is about what we expected**

now if you want your results in a single array you can concatenate on the time dimension

In [13]:
lc_array=xr.concat(result,dim='time')
lc_array

<xarray.DataArray 'lccs_class' (time: 11, y: 7673, x: 9902)>
array([[[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
...
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]]], dtype=uint8)
Coordinates:
  * x            (x) float64 -2.828e+06 -2.828e+06 ... 7.072e+06 7.072e+06
  * y            (y) float64 3.168e+06 3.167e+06 ... -4.503e+06 -4.504e+06
  * time         (time) datetime64[ns] 2016-01-01 2014-01-01 ... 2010-01-01
    spatial_ref  int64 0
Attributes:
    standard_name:        land_cover_lccs
    flag_colors:          #ffff64 #ffff64 #ffff00 #aaf0f0 #dcf064 #c8c864 #00...
    long_name:            Land cover class defined in LCCS
    valid_min:            1
    valid_max:            220
    ancillary_variables:  processed_flag current_pixel_state observation_coun...
    flag_meanings:        no_data cropland_rainfed cropland_rainfed_herbaceou...
    flag_values:          [  0  10  11  12  20  30  40  50  60  61  62  70  7...
    _FillValue:           255

**but notice the times aren't in order because the files weren't in order**

In [14]:
# this will sort time ascending
lc_array=lc_array.sortby(lc_array.time)
lc_array

<xarray.DataArray 'lccs_class' (time: 11, y: 7673, x: 9902)>
array([[[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
...
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]]], dtype=uint8)
Coordinates:
  * x            (x) float64 -2.828e+06 -2.828e+06 ... 7.072e+06 7.072e+06
  * y            (y) float64 3.168e+06 3.167e+06 ... -4.503e+06 -4.504e+06
  * time         (time) datetime64[ns] 2010-01-01 2011-01-01 ... 2020-01-01
    spatial_ref  int64 0
Attributes:
    standard_name:        land_cover_lccs
    flag_colors:          #ffff64 #ffff64 #ffff00 #aaf0f0 #dcf064 #c8c864 #00...
    long_name:            Land cover class defined in LCCS
    valid_min:            1
    valid_max:            220
    ancillary_variables:  processed_flag current_pixel_state observation_coun...
    flag_meanings:        no_data cropland_rainfed cropland_rainfed_herbaceou...
    flag_values:          [  0  10  11  12  20  30  40  50  60  61  62  70  7...
    _FillValue:           255